In [26]:
import pandas as pd 
import numpy as np
pd.set_option('display.max_colwidth', None)

In [27]:
# Tratamento de warning
import warnings
warnings.filterwarnings('ignore')

In [28]:
%store -r datatran

In [29]:
df = datatran

### Análise da causa do acidente, coluna de texto através de Topic Modeling

In [30]:
df['causa_acidente'].unique()

array(['Manobra de mudança de faixa',
       'Acessar a via sem observar a presença dos outros veículos',
       'Reação tardia ou ineficiente do condutor',
       'Ausência de reação do condutor', 'Frear bruscamente',
       'Demais falhas mecânicas ou elétricas', 'Transitar na contramão',
       'Condutor Dormindo', 'Velocidade Incompatível',
       'Entrada inopinada do pedestre',
       'Objeto estático sobre o leito carroçável',
       'Acumulo de água sobre o pavimento', 'Pedestre andava na pista',
       'Ultrapassagem Indevida',
       'Trafegar com motocicleta (ou similar) entre as faixas',
       'Condutor deixou de manter distância do veículo da frente',
       'Pedestre cruzava a pista fora da faixa',
       'Ingestão de álcool pelo condutor', 'Problema com o freio',
       'Acesso irregular', 'Retorno proibido', 'Pista Escorregadia',
       'Desrespeitar a preferência no cruzamento', 'Pista esburacada',
       'Conversão proibida', 'Avarias e/ou desgaste excessivo no pneu'

### Tokenização para remoção de stopwords

In [31]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [8]:
# # Baixar dados necessários do NLTK
# nltk.download('punkt')
# nltk.download('stopwords')

In [32]:
custom_stopwords = set(stopwords.words('portuguese')) | set([
    " ", "ou", "", "ok", "problema", "paciente", "necessário", "feito", "w", "devido",
    "precisa", "dr", "opções", "caso", "o", "x", "tentar", "ainda", "próximo", "r",
    "d", "desde", "teste", "testando", "resultados", "recomendar", "pode", "por favor",
    "médico", "normal", "seria", "discutido", "medicamento", "doença",
    "sugerido", "considerar", "sim", "provável", "clínico", "revisão",
    "interno", "tratar", "rever", "semana", "ensaio", "comentário", "rec",
    "vai", "nós", "oi", "olá", "cumprimento", "gostar", "saber", "sim", "certo",
    "amanhã", "olhar", "dizer", "okay", "quilograma", "zoom", "link", "tudo bem"
])


In [33]:
# Tokenizar e remover stopwords
def process_text(text):
        tokens = word_tokenize(str(text).lower())  # Tokenizar e converter para minúsculas
        filtered_tokens = [word for word in tokens if word.isalnum() and word not in custom_stopwords]
        return " ".join(filtered_tokens)  # Reunir palavras filtradas em uma string

# Aplicar processamento ao DataFrame
df["palavras_filtradas"] = df["causa_acidente"].apply(process_text)

Lematização e Radicalização

🔹 Lema (Lemmatization): Processo de reduzir palavras à sua forma base ou raiz, conhecida como lema.

🔹 Radical (Stemming): Redução de palavras à sua forma raiz (stem) sem considerar regras gramaticais ou significado.



In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def lemma_nltk(df):
    # Inicializar o the lemmatizer
    lemmatizer = WordNetLemmatizer()

    def get_wordnet_pos(word):
        """Map POS tag to first character lemmatize() accepts"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)

    def lemmatize_words(text):
        words = text.split()
        lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in words]
        return ' '.join(lemmatized_words)

    # Aplicar lemmatization function a coluna do DataFrame
    df['lemma'] = df['palavras_filtradas'].apply(lemmatize_words)

    return df

In [36]:
%%time
df_lemma = lemma_nltk(df)

Wall time: 32.7 s


In [37]:
df_lemma[['palavras_filtradas', 'lemma']].head(10)

,palavras_filtradas,lemma
0,manobra mudança faixa,manobra mudança faixa
1,acessar via observar presença outros veículos,acessar via observar presença outros veículos
2,reação tardia ineficiente condutor,reação tardia ineficiente condutor
3,reação tardia ineficiente condutor,reação tardia ineficiente condutor
4,reação tardia ineficiente condutor,reação tardia ineficiente condutor
5,ausência reação condutor,ausência reação condutor
6,manobra mudança faixa,manobra mudança faixa
7,frear bruscamente,frear bruscamente
8,reação tardia ineficiente condutor,reação tardia ineficiente condutor
9,ausência reação condutor,ausência reação condutor


In [ ]:
from nltk.stem import PorterStemmer

def stem_nltk_after_lemma(df):
    # Inicializar o stemmer
    stemmer = PorterStemmer()

    def stem_words(text):
        # Split do texto em palavras
        words = text.split()
        # Stem cada palavra
        stemmed_words = [stemmer.stem(word) for word in words]
        # Join as palavras apos o stemmed words de volta a uma single string
        return ' '.join(stemmed_words)

    # Aplica a stemming function ao DataFrame
    df['normalizado'] = df['lemma'].apply(stem_words)

    return df

In [39]:
%%time
df_final = stem_nltk_after_lemma(df_lemma)

Wall time: 1.27 s


In [40]:
df_final[['palavras_filtradas', 'lemma', 'normalizado']].head(10)

,palavras_filtradas,lemma,normalizado
0,manobra mudança faixa,manobra mudança faixa,manobra mudança faixa
1,acessar via observar presença outros veículos,acessar via observar presença outros veículos,acessar via observar presença outro veículo
2,reação tardia ineficiente condutor,reação tardia ineficiente condutor,reação tardia ineficient condutor
3,reação tardia ineficiente condutor,reação tardia ineficiente condutor,reação tardia ineficient condutor
4,reação tardia ineficiente condutor,reação tardia ineficiente condutor,reação tardia ineficient condutor
5,ausência reação condutor,ausência reação condutor,ausência reação condutor
6,manobra mudança faixa,manobra mudança faixa,manobra mudança faixa
7,frear bruscamente,frear bruscamente,frear bruscament
8,reação tardia ineficiente condutor,reação tardia ineficiente condutor,reação tardia ineficient condutor
9,ausência reação condutor,ausência reação condutor,ausência reação condutor


In [41]:
df_final[df_final['lemma']==''].head(5)

,id,ano,feriado,mes,data_inversa,uf,br,km,municipio,causa_acidente,...,mortos,feridos_graves,latitude,longitude,status,regiao,periodo,palavras_filtradas,lemma,normalizado


### LDA (entendendo o algoritmo)

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import gensim
from gensim.models import CoherenceModel
from gensim.corpora.dictionary import Dictionary

In [43]:

def lda(df_final, topics, words, threshold):
    """
    Executa a **Latent Dirichlet Allocation (LDA)** em um DataFrame para identificar tópicos.

    **Parâmetros:**
    df_final (DataFrame): O DataFrame de entrada contendo os dados de texto.
    topics (int): O número de tópicos a identificar.
    words (int): O número de palavras mais relevantes a serem exibidas para cada tópico.

    **Retorna:**
    df_topics (DataFrame): O DataFrame de saída contendo os dados de texto.

    Esta função divide o DataFrame em conjuntos de treino e teste, **vetoriza os dados de texto** usando **TF-IDF**, 
    ajusta um modelo LDA nos dados de treino e transforma os dados de teste. 
    Ela imprime o número de linhas nos conjuntos de treino e teste, confirma a criação das matrizes correspondentes, 
    e indica a conclusão das fases de treinamento e teste.
    """

    # Separar o DataFrame em conjuntos de treino e teste
    train_df = df_final.sample(frac=0.7, random_state=42)
    test_df = df_final.drop(train_df.index)

    print(f"Contagem df treino: {len(train_df)}")
    print(f"Contagem df teste: {len(test_df)}")

    train_texts = train_df['normalizado'].tolist()
    test_texts = test_df['normalizado'].tolist()

    vectorizer = TfidfVectorizer()
    train_matrix = vectorizer.fit_transform(train_texts)
    test_matrix = vectorizer.transform(test_texts)

    print(f"Matrix de treino e testes criadas!")

    # Define o número de tópicos e palavras dentro de cada tópico
    num_topics = topics
    num_top_words = words

    # Criação LDA object
    lda = LatentDirichletAllocation(doc_topic_prior=0.5, learning_decay=0.5, learning_method='online', max_iter= 10, topic_word_prior=0.1, n_components=num_topics, random_state=42)

    # Fit do modelo na matrix de treino
    lda_matrix_train = lda.fit_transform(train_matrix)

    print(f"Treinamento completo!")

    # Transforma a matrix de teste
    lda_matrix_test = lda.transform(test_matrix)

    print(f"Teste completo!")

    # Obtém os termos de cada tópico
    terms = vectorizer.get_feature_names_out()

    topics = []
    top_terms_dict = {}
    for idx, topic in enumerate(lda.components_):
        top_terms = [terms[i] for i in topic.argsort()[-num_top_words:]]
        topics.append((f"Topic {idx + 1}", ", ".join(top_terms)))
        top_terms_dict[idx] = ", ".join(top_terms)

    # Criação df com os tópicos
    df_topics = pd.DataFrame(topics, columns=["Topic", "Top Terms"])

    print(f"Tópicos gerados!")

    # Teste de perplexidade
    perplexity = lda.perplexity(test_matrix)
    print(f'Perplexity: {perplexity}')
 
    # Prepara o dado para coherence model
    texts = [doc.split() for doc in test_texts]
    dictionary = Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    # Fit do modelo LDA usando gensim
    lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

    # Cálculo coherence score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model_lda.get_coherence()
    print(f'Coherence Score: {coherence_score}')

    # Atribue os tópicos para o df original
    # O parâmetro de limite (threshold) é utilizado para garantir que apenas tópicos com uma probabilidade acima do limite especificado sejam atribuídos. Se a maior probabilidade estiver abaixo desse limite, o tópico é definido como -1, indicando que não há uma atribuição clara de tópico. Essa abordagem pode ajudar a melhorar a precisão do mapeamento de tópicos.
    num_threshold = threshold
    topic_prob_matrix = lda.transform(vectorizer.transform(df_final['normalizado'].tolist()))
    df_final['Topic'] = np.where(topic_prob_matrix.max(axis=1) >= num_threshold, topic_prob_matrix.argmax(axis=1), -1)
    df_final['Top Terms'] = df_final['Topic'].map(lambda x: top_terms_dict.get(x, ''))

    return df_topics, df_final

In [47]:
%%time
df_topics, df_final = lda(df_final, 8, 6, 0.1)

Contagem df treino: 13996
Contagem df teste: 5999
Matrix de treino e testes criadas!
Treinamento completo!
Teste completo!
Tópicos gerados!
Perplexity: 47.724964704056994
Coherence Score: 0.5833879736271982
Wall time: 1min 9s


In [49]:
df_final[['Topic', 'Top Terms']].drop_duplicates().sort_values(by='Topic', ascending=True)

,Topic,Top Terms
16,0,"trânsito, largura, insuficient, modificação, velocidad, incompatível"
0,1,"trafegar, similar, motocicleta, mudança, manobra, faixa"
2,2,"chuva, mal, condutor, reação, ineficient, tardia"
1,3,"outro, observar, acessar, presença, via, veículo"
17,4,"andava, cruzamento, preferência, desrespeitar, pedestr, pista"
5,5,"mecânica, falha, demai, condutor, reação, ausência"
12,6,"sobr, acostamento, ultrapassagem, indevida, contramão, transitar"
14,7,"animai, freio, álcool, ingestão, dormindo, condutor"


In [50]:
topico_para_categoria = {
    0: "Infração de Trânsito",
    1: "Obstáculo ou Motocicleta envolvida",
    2: "Condutor reação tardia",
    3: "Manobra imprudente condutor",
    4: "Manobra imprudente condutor",
    5: "Problema mecânico no veículo",
    6: "Acostamento ou Ultrapassagem",
    7: "Condutor sob efeito de substâncias",
}

Explorando os tópicos

In [51]:
df_final[df_final['normalizado'].str.contains('observar', na=False)][['causa_acidente','Topic', 'Top Terms', 'normalizado']].drop_duplicates()

,causa_acidente,Topic,Top Terms,normalizado
1,Acessar a via sem observar a presença dos outros veículos,3,"outro, observar, acessar, presença, via, veículo",acessar via observar presença outro veículo


In [52]:
df_final[df_final['normalizado'].str.contains('ineficient', na=False)][['Topic', 'Top Terms', 'normalizado']].drop_duplicates()

,Topic,Top Terms,normalizado
2,2,"chuva, mal, condutor, reação, ineficient, tardia",reação tardia ineficient condutor
1710,4,"andava, cruzamento, preferência, desrespeitar, pedestr, pista",sistema drenagem ineficient


### Analisando Medidas de Acurácia

#### 1. Perplexity
A perplexidade é uma medida de quão bem um modelo probabilístico prevê um conjunto de amostras. No contexto de LDA (Latent Dirichlet Allocation), valores mais baixos de perplexidade indicam um melhor ajuste aos dados.


#### 2. Coherence Score
A coerência mede a similaridade semântica entre as palavras mais relevantes dentro dos tópicos. Pontuações mais altas indicam tópicos de melhor qualidade. 

Um Coherence Score de 0.5329 indica que os tópicos gerados pelo modelo LDA possuem um nível razoável de interpretabilidade e similaridade semântica.
Interpretação da pontuação:

- Acima de 0.5 → Indica que os tópicos têm uma coerência moderada a boa, com palavras que fazem sentido juntas.
- Entre 0.6 e 0.8 → Indica boa coerência semântica, sugerindo que os tópicos estão bem agrupados e interpretáveis.
- Acima de 0.8 → Indica alta qualidade, onde os tópicos gerados são claramente diferenciáveis e têm forte significado semântico.

🔹 Resultado (0.5329) sugere que os tópicos fazem sentido, mas ainda há espaço para refinamento. Algumas melhorias que podem ser feitas para aumentar a coerência:

- Refinar o pré-processamento do texto → Remover palavras irrelevantes ou normalizar melhor o texto.
- Ajustar o número de tópicos → Talvez reduzir ou aumentar um pouco os tópicos ajude na segmentação das palavras.
- Alterar hiperparâmetros do LDA → Testar ajustes na alpha e beta, além de aumentar o número de iterações.


### GRID SEARCH 

>- simplificado devido à problemas de performance do micro

In [53]:
from utils.grid_search import grid_search

In [54]:
%%time
df_topics, df_final = grid_search(df_final)

Contagem df treino: 13996
Contagem df teste: 5999
Matrix de treino e testes criadas!
Fitting 2 folds for each of 32 candidates, totalling 64 fits
[CV] END doc_topic_prior=0.1, learning_decay=0.3, learning_method=batch, max_iter=5, n_components=5, topic_word_prior=0.01; total time=   2.7s
[CV] END doc_topic_prior=0.1, learning_decay=0.3, learning_method=batch, max_iter=5, n_components=5, topic_word_prior=0.01; total time=   2.7s
[CV] END doc_topic_prior=0.1, learning_decay=0.3, learning_method=batch, max_iter=5, n_components=5, topic_word_prior=0.1; total time=   2.8s
[CV] END doc_topic_prior=0.1, learning_decay=0.3, learning_method=batch, max_iter=5, n_components=5, topic_word_prior=0.1; total time=   2.7s
[CV] END doc_topic_prior=0.1, learning_decay=0.3, learning_method=batch, max_iter=5, n_components=8, topic_word_prior=0.01; total time=   2.8s
[CV] END doc_topic_prior=0.1, learning_decay=0.3, learning_method=batch, max_iter=5, n_components=8, topic_word_prior=0.01; total time=   3.3

In [55]:
df_final[['Topic', 'Top Terms']].drop_duplicates().sort_values(by='Topic', ascending=True)

,Topic,Top Terms
2,0,"largura, curva, acentuada, incompatível, velocidad, tardia, ineficient, ausência, condutor, reação"
0,1,"cruzamento, preferência, desrespeitar, trafegar, similar, motocicleta, freio, mudança, manobra, faixa"
14,2,"carga, súbito, chuva, mal, álcool, ingestão, indevida, ultrapassagem, condutor, dormindo"
1,3,"presença, acessar, outro, observar, mecânica, elétrica, via, demai, falha, veículo"
12,4,"pneu, desgast, avaria, excessivo, animai, andava, acostamento, pista, contramão, transitar"


In [56]:
topico_para_categoria = {
    0: "Condutor reação tardia",
    1: "Infração de Trânsito",
    2: "Condutor sob efeitos de substâncias",
    3: "Problema mecânico no veículo",
    4: "Manobra imprudente condutor"
}

In [57]:
df_final['Categoria'] = df_final['Topic'].map(topico_para_categoria)

In [58]:
df_final[['Categoria', 'Topic', 'Top Terms', 'normalizado']]

,Categoria,Topic,Top Terms,normalizado
0,Infração de Trânsito,1,"cruzamento, preferência, desrespeitar, trafegar, similar, motocicleta, freio, mudança, manobra, faixa",manobra mudança faixa
1,Problema mecânico no veículo,3,"presença, acessar, outro, observar, mecânica, elétrica, via, demai, falha, veículo",acessar via observar presença outro veículo
2,Condutor reação tardia,0,"largura, curva, acentuada, incompatível, velocidad, tardia, ineficient, ausência, condutor, reação",reação tardia ineficient condutor
3,Condutor reação tardia,0,"largura, curva, acentuada, incompatível, velocidad, tardia, ineficient, ausência, condutor, reação",reação tardia ineficient condutor
4,Condutor reação tardia,0,"largura, curva, acentuada, incompatível, velocidad, tardia, ineficient, ausência, condutor, reação",reação tardia ineficient condutor
...,...,...,...,...
19993,Problema mecânico no veículo,3,"presença, acessar, outro, observar, mecânica, elétrica, via, demai, falha, veículo",acessar via observar presença outro veículo
19994,Condutor sob efeitos de substâncias,2,"carga, súbito, chuva, mal, álcool, ingestão, indevida, ultrapassagem, condutor, dormindo",ingestão álcool condutor
19995,Infração de Trânsito,1,"cruzamento, preferência, desrespeitar, trafegar, similar, motocicleta, freio, mudança, manobra, faixa",freio
19996,Problema mecânico no veículo,3,"presença, acessar, outro, observar, mecânica, elétrica, via, demai, falha, veículo",condutor deixou manter distância veículo frent


### Salvando para o PowerBI

In [59]:
df_final.to_csv('./resultados/Data categorizado.csv', index=False, sep=';')